<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/topology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algebraic Topology (Persistent Homology)**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

![Normed Vector Space](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/P1S2all.jpg/798px-P1S2all.jpg)

## **From Quotient Groups to Simplicial Homology**

#### **Quotient Groups**

**Division mit Rest & Modulo Operation (Restklassenrechnung)**

* **Division mit Rest** (Divisionsalgorithmus) besagt, dass es zu zwei Zahlen $n$ und $m \neq 0$ eindeutig bestimmte Zahlen $a$ und $b$ gibt, für die $n=m \cdot a+b, \quad 0 \leq b<|m|$ gilt. ZB 17 = 3 * 5 + 2 (b ist hierbei die Restklasse und a der Quotient). Sources: [Division_mit_Rest](https://de.wikipedia.org/wiki/Division_mit_Rest) & [Euclidean_division](https://en.wikipedia.org/wiki/Euclidean_division)

* **Modulo (Restklassenrechnung) berechnet den Rest b der Division n geteilt durch m ('Modular arithmetic')**. Source: [Modulo Operation](https://en.m.wikipedia.org/wiki/Modulo_operation) & [Division_mit_Rest#Modulo](https://de.m.wikipedia.org/wiki/Division_mit_Rest#Modulo). Beispiel: 7 mod 2 (sprich 7modulo2) ist beispielsweise 1, denn 7 geteilt durch 2 lässt den Rest 1.
  * Zwei Zahlen sind kongruent (modulo der natürlichen Zahl n), wenn ihre Differenz durch n teilbar ist.
  * Zwei Zahlen sind kongruent (modulo des Moduls m), wenn ihre Differenz durch m teilbar ist.
  * Die ganzen Zahlen zerfallen mit der Kongruenz also in Äquivalenzklassen, die Restklassen modulo m.

* **Äquivalenzklassen**: The result of the modulo operation is an equivalence class (Äquivalenzklassen). Any member of the class may be chosen as representative. However, the usual representative is the least positive residue, the smallest non-negative integer that belongs to that class (i.e., the remainder of the [Euclidean division](https://en.wikipedia.org/wiki/Euclidean_division) / Division mit Rest) & [Equivalence_class](https://en.m.wikipedia.org/wiki/Equivalence_class)

* **[Restklassen](https://de.wikipedia.org/wiki/Restklasse)** sind die Äquivalenzklassen in der Kongruenzrelation. Eine Zahl a modulo einer Zahl m die Menge aller Zahlen, die bei Division durch m denselben Rest lassen wie a. Jede Menge $\bar{a}=\{x \in \mathbb{Z} \mid x \equiv a$ mod $m\}$ nennt man eine **Restklasse
modulo $m .$** Jedes Element $x \in \bar{a}$ heißt **Repräsentant** der Restklasse $\bar{a}$. Die Menge aller Restklassen modulo m bezeichnet man mit $\mathrm{R}_{\mathrm{m}}$. Rechnen mit Restklassen: Mit Hilfe der **[Rechengesetze für Kongruenzen](https://de.wikipedia.org/wiki/Kongruenz_(Zahlentheorie)#Rechenregeln)** können wir nun überlegen, ob wir auf einfache Weise auch die Summe zweier Zahlen einer Restklasse zuordnen können. Zum Beispiel direkt über die Addition der Restklassen. In der Gruppentheorie werden Äquivalenzklassen als [Nebenklassen (Cosets)](https://de.m.wikipedia.org/wiki/Gruppentheorie#Nebenklassen) bezeichnet

**Äquivalenz & Kongruenz**

* **Äquivalenz**: Objekte, die sich in einem bestimmten Zusammenhang gleichen, als gleichwertig bzw. äquivalent angesehen. Ein solcher Zusammenhang lässt sich für alle Elemente einer nichtleeren Menge $A$ stets durch eine Funktion $f: A \rightarrow B$ herstellen, indem man genau dann zwei Elemente $a, b \in A$ als zueinander $_{n}$ äquivalent $^{\prime \prime}$ bezeichnet und diese Beziehung durch $a \sim b$ symbolisiert, wenn deren Bilder gleich sind: $
a \sim b: \Longleftrightarrow f(a)=f(b)
$. Diese Beziehung bzw. Relation hat die folgenden drei Eigenschaften:
  1. Jedes Objekt $a$ ist zu sich selbst äquivalent: $a \sim a$.
  2. Wenn $a$ äquivalent zu $b$ ist, dann ist auch $b$ äquivalent zu $a: a \sim b \Longrightarrow b \sim a$.
  3. Wenn $a$ äquivalent zu $b$ und $b$ äquivalent zu $c$ ist, dann ist auch $a$ äquivalent zu $c: a \sim b$ und $b \sim c \Longrightarrow a \sim c$.
  
* **Kongruenz**: ([Kongruenz (Geometrie)](https://de.m.wikipedia.org/wiki/Kongruenz_(Geometrie)) bzw. [Kongruenz (Zahlentheorie)](https://de.m.wikipedia.org/wiki/Kongruenz_(Zahlentheorie))) Die Kongruenz zwischen zwei ganzen Zahlen ist in Bezug auf einen Teiler definiert. **Der Teiler heißt in diesem Zusammenhang Modul.** Gegeben sei ein Modul $m \in \mathbb{N}$. Zwei ganze Zahlen $a$ und $b$ heißen kongruent modulo $m$, **wenn die Division von $a$ und $b$ durch $m$ den gleichen Rest $r$ lässt**. Kongruenz ist eine Art Erweiterung der Modulorechnung:
  * Modulorechnung: 17 mod 3 = 2. 
  * **Kongruenz: 11 $\equiv$ 17 mod 3 (weil bei beiden der Rest 2 ist - beide sind in der der gleichen Restklasse)**
  * Man kann alternativ zur Restklassenermittlung auch sagen, dass zwei Zahlen kongruent sind (modulo der natürlichen Zahl n), wenn ihre Differenz (hier: h(e) - t(e) ) durch n teilbar ist. Hier: 17 - 11 = 6, ist teilbar durch 3.
  * $
a \equiv b \quad(\bmod m) \quad \Leftrightarrow \quad(a \bmod m)=(b \bmod m) \quad \Leftrightarrow \quad(a-b) \bmod m=0
$
  * $a \equiv b \bmod m \Leftrightarrow \exists t_{a}, t_{b}, r \in \mathbb{Z}: a=t_{a} m+r$ und $b=t_{b} m+r$ und $0 \leq r<m$
  * Jede Kongruenz modulo einer ganzen Zahl ist eine Kongruenzrelation auf dem Ring der ganzen Zahlen.
  * **15 is congruent to 3 modulo 12**, so "15:00" on a 24-hour clock is displayed "3:00" on a 12-hour clock.

* **Rechengesetze für Kongruenzen**: Kongruenzen darf man wie Gleichungen mit Rechnungen kombinieren. Dabei ist allerdings die Division nicht zulässig, man muss sich auf +, – und · beschränken.

  * Sei $m \in \mathbb{N}$ ein Modul und seien $a, b, c, d \in \mathbb{Z}$ $a \equiv b \bmod m$ und $c \equiv d \bmod m \Rightarrow\left\{\begin{array}{l}a+c \equiv b+d \bmod m \\ a-c \equiv b-d \bmod m \\ a \cdot c \equiv b \cdot d \bmod m\end{array}\right.$

**Äquivalenz- & Kongruenzrelation**

* Siehe auch: Ordnungstheorie -> Typen von Ordnungsrelationen

* Eine **Äquivalenzrelation** auf einer Menge $A \neq \emptyset$ ist eine zweistellige Relation $\sim \subseteq A \times A,$ die folgende Bedingungen erfüllt:
  * **Reflexivität**
$
(a, a) \in \sim \text { für alle } a \in A
$
  * **Symmetrie** $(a, b) \in \sim \Longrightarrow(b, a) \in \sim$ für alle $a, b \in A$
  * **Transitivität** $(a, b) \in \sim$ und $(b, c) \in \sim \Longrightarrow(a, c) \in \sim$ für alle $a, b, c \in A$
* Wie bei zweistelligen Relationen üblich, schreibt man statt $(a, b) \in \sim$ auch einfacher $a \sim b,$ dann nehtnen diese Eigenschaften die oben genannte Form an. Das geordnete Paar $(A, \sim)$ nennt man in diesem Fall auch Setoid oder E-set (englische Bezeichnung: extensional set, auch Bishop set). Eine Äquivalenzrelation ist somit eine zweistellige Relation, die **reflexiv, symmetrisch und transitiv** ist. **Eine Äquivalenzrelation teilt eine Menge restlos in disjunkte (elementfremde) Untermengen**, Äquivalenzklassen genannt. Die Klassenbildung mit Hilfe des Äquivalenzbegriffes ist grundlegend für viele mathematische Begriffsbildungen.
* **Die [Kongruenzrelation](https://de.m.wikipedia.org/wiki/Kongruenzrelation) {[Algebraische_Struktur#Kongruenzrelationen](https://de.m.wikipedia.org/wiki/Algebraische_Struktur#Kongruenzrelationen) } ist für alle Modulo m auf der Menge $Z$ eine Äquivalenzrelation.** Die Kongruenzrelation ist letztlich deshalb eine Äquivalenzrelation, weil die grundlegende Eigenschaft „haben den gleichen Teilungsrest“ auf die Gleichheit zurückgreift, die eine Äquivalenzrelation ist. 
* In der Algebra **nennt man eine Äquivalenzrelation auf einer algebraischen Struktur eine Kongruenzrelation**, wenn die fundamentalen Operationen der algebraischen Struktur mit dieser Äquivalenzrelation verträglich sind. Eine Kongruenzrelation ist daher eine spezielle Äquivalenzrelation. Sie hat also die folgenden Eigenschaften:
  1. Reflexivität
$a \equiv a \quad(\bmod m)$ für alle $a \in \mathbb{Z}$
  2. Symmetrie $a \equiv b \quad(\bmod m) \Rightarrow b \equiv a \quad(\bmod m)$ für alle $a, b \in \mathbb{Z}$
  3. Transitivität
$a \equiv b \quad(\bmod m)$ und $b \equiv c \quad(\bmod m) \Rightarrow a \equiv c \quad(\bmod m)$ für alle $a, b, c \in \mathbb{Z}$
* Eine **Äquivalenzrelation auf einer Menge $A$ hat nicht notwendigerweise etwas mit der Struktur zu tun**, die darauf definiert ist. Von besonderem Interesse sind jedoch solche Äquivalenzrelationen $\equiv$ , deren Quotientenabbildung $
\mathrm{q}_{\mathrm{z}}: A \rightarrow A / \equiv, a \mapsto[a]_{\equiv}
$ **mit der Struktur auf $A$ verträglich bzw. ein Homomorphismus ist**, weil dann die von $\mathrm{q}_{=}$ erzeugte Struktur auf der [Quotientenmenge](https://de.wikipedia.org/wiki/Äquivalenzrelation#Quotientenmenge_und_Partition) $A / \equiv$ von der gleichen Art ist wie die von $A$. **Eine solche Äquivalenzrelation $\equiv$ nennt man eine Kongruenzrelation auf der strukturierten Menge $A$.**
  * Die Quotientengruppe G/N ist homomorph zur Gruppe G ( ??? )
* Insbesondere sind dann auch alle zur Struktur gehörenden Funktionen mit $\equiv$ verträglich.

**Groups & Subgroups**

* Sei $\mathbb{Z}$ die Gruppe der ganzen Zahlen mit der Addition als Gruppenoperation.

* Man kann fur diese Gruppe verschiedene (unendlich viele) **[Untergruppen](https://de.m.wikipedia.org/wiki/Untergruppe) (Subgroups)** bilden: 2$\mathbb{Z}$, 3$\mathbb{Z}$, 4$\mathbb{Z}$, 5$\mathbb{Z}$, 6$\mathbb{Z}$, 7$\mathbb{Z}$, ..

* Additional Information:

  * A subgroup is a subset that‘s also a group, like H $\leq$ G. And a "proper subset" is: H < G. (Beispiel: Die ganzen Zahlen $\mathbb {Z} $ sind bezüglich der Addition eine Untergruppe der rationalen Zahlen $\mathbb {Q} $.)

  * Two standard subgroups of G are G itself and the Trivial Group = {e} with one element only, which is the identity element. Every group has at least 2 sub groups: the identity element {e} and the entire group G (these are technically normal subgroups, but aren't very interesting). If a group has no other normal subgroups then these two, than it's called a **simple group**. A simple group does not have any factor (quotient) groups, but they are the building blocks of other groups.

  * **Langrange‘s theorem**: order of G is the number of elements in G, or: if H $\leq$ G, then the order of H divides the order of G = |G|. Nebenklassen werden benutzt, um den [Satz von Lagrange](https://de.m.wikipedia.org/wiki/Satz_von_Lagrange) zu beweisen, um die Begriffe [Normalteiler](https://de.m.wikipedia.org/wiki/Normalteiler) und [Faktorgruppe](https://de.m.wikipedia.org/wiki/Faktorgruppe) zu erklären und um Gruppenoperationen zu studieren.

**Normal Subgroups (Normalteiler)**

* Die Gruppe $\mathbb{Z}$ ist **abelsch** (kommutativ) und somit ist jede Untergruppe auch ein **[Normalteiler](https://de.wikipedia.org/wiki/Normalteiler) bzw. [Normal Subgroup](https://en.wikipedia.org/wiki/Normal_subgroup)**.

* WICHTIG: Normalteiler sind spezielle Untergruppen, und ihre Bedeutung liegt vor allem darin, **dass sie genau die Kerne von Gruppenhomomorphismen sind**. Normal Subgroups heissen auch "invariant or self-conjugate subgroups".

  * Kern: Lösungsmenge der homogenen linearen Gleichung f(x) = 0.

* Additional Information:

  * Ein Normalteiler N < G ist eine Untergruppe mit der speziellen Eigenschaft aN=Na für alle a in G.

  * Normal subgroups and quotient groups are among the most useful devices in abstract algebra. **Normal subgroups determine what kinds of homomorphisms are possible from a group G to other groups (f : G -> H)**

  * Die Bezeichnung „…teiler“ in Normalteiler bezieht sich darauf, dass sich aus einer Gruppe G und jedem ihrer Normalteiler N eine Faktorgruppe G/N bilden lässt. Diese [Faktorgruppen](https://de.wikipedia.org/wiki/Faktorgruppe) sind homomorphe Bilder von G, **und jedes homomorphe Bild von G ist zu einer solchen Faktorgruppe G/N isomorph**.

    *  ( ?? ) See fundamental theorem on homomorphisms of groups as "**Every homomorphic image of a group is isomorphic to a quotient group**". [Source](https://en.m.wikipedia.org/wiki/Fundamental_theorem_on_homomorphisms)

    * ![Image](https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Fundamental_Homomorphism_Theorem.svg/210px-Fundamental_Homomorphism_Theorem.svg.png)

  * And for finite groups you can find a chain of normal subgroups called a "composition series" which acts as a kind of 'prime factorization' of the group (1 ◃ N1, ◃ N2 ◃ ... ◃ Nr ◃ G ). Normal subgroups can also be used to study fields (Körper), i.e. in Galois theory (Field extension K / F)

**Congruence Classes & Cosets (Nebenklassen)**

* Nehmen wir jetzt die Untergruppe 5$\mathbb{Z}$ von $\mathbb{Z}$ (integers mod 5), die aus allen Vielfachen von 5 besteht (the subgroup of the multiples of 5).

* Wir bilden jetzt die **Congruence Classes** (Äquivalenzklassen) aus den 5 Sets der Reste (remainders) when dividing by 5 (remainder 0, 1, 2, 3, 4). Technically we use the normal subgroup to divide a group into **cosets**:

  * 5$\mathbb{Z}$ + Rest 0 = $\overline{0} : \{\ldots-10,-5,0,5,10 \ldots\}$ **(Normal) Subgroup**, which is technically also a Coset 0+ 5 $\mathbb{Z}$

  * 5 $\mathbb{Z}$ + Rest 1 = $\overline{1} : \{\ldots-9,-4,1,6,11 \ldots\}$ **Coset 1+ 5 $\mathbb{Z}$**

  * 5 $\mathbb{Z}$ + Rest 2 = $\overline{2} : \{\ldots-8,-3,2,7,12 \ldots\}$ **Coset 2+ 5 $\mathbb{Z}$**

  * 5 $\mathbb{Z}$ + Rest 3 = $\overline{3} : \{\ldots-7,-2,3,8,13 \ldots\}$ **Coset 3+ 5 $\mathbb{Z}$**

  * 5 $\mathbb{Z}$ + Rest 4 = $\overline{4} : \{\ldots-6,-1,4,9,14 \ldots\}$ **Coset 4+ 5 $\mathbb{Z}$**

* Die Congruence Classes mit einem Rest anders als 0 (also 1, 2, 3 und 4), sind **keine Subgroups**, sondern nur **Coset (Nebenklasse)**, weil: it's not closed under addition, has no inverse and doesn't contain the identity element. (**Difference between Subgroup and subset: Subgroup has the idenity element**)

* Additional Information:
  * Cosets are there to define how many (finite) possible subgroup exist in a group
  * There are **two types of cosets**: left cosets and right cosets (linke und rechte Nebenklasse). Cosets (of either type) have the same number of elements (cardinality) as does H. 
  * Furthermore, H itself is a coset, which is both a left coset and a right coset. The number of left cosets of H in G is equal to the number of right cosets of H in G. The common value is called the index of H in G and is usually denoted by [G : H].

**Quotient Group (Faktorgruppe)**

 * If we treat the 5 sets of remainders (the Congruence classes) **as if they were numbers**, then we have **a group with 5 elements**, the **Quotient Group** (Faktorgruppe)

  * $\mathbb{Z}$ mod 5 = {$\overline{0}$, $\overline{1}$, $\overline{2}$, $\overline{3}$, $\overline{4}$}

* Quotient Group besteht die aus allen **Nebenklassen (Cosets)** der Untergruppe 5$\mathbb{Z}$. Man sagt auch: The group of cosets form the quotient group, and we can treat these cosets as elements in a new group: the **coset group**.

* Diese Quotient Group bildet wiederum selbst eine **group under addition**.
 * The set of the multiples of 5 act as the idenityt element = $\overline{0}$ 
 * and each set has an inverse
 * Each set is closed under +

* However, the quotient group is not a subgroup of $\mathbb{Z}$. It's an entirely different group.

* Additional Information:

  * **Die Faktorgruppe oder Quotientengruppe ist eine Gruppe, die mittels einer Standardkonstruktion aus einer gegebenen Gruppe $G$ unter Zuhilfenahme eines Normalteilers $N \leq G$ gebildet wird. Sie wird mit $G / N$ bezeichnet und ist die Menge der Nebenklassen (Cosets).** Sie unterteilt eine Menge in Äquivalenzklassen bzw. eine Gruppe in Restklassen / Nebenklassen.

  * Die **Menge der Restklassen (Äquivalenzklassen) heisst Quotientenmenge bzw. Faktormenge**. In der Gruppentheorie heissen Restklassen 'Nebenklassen' (Cosets) und die Menge der Nebenklassen zu einer Gruppe heißt Faktorgruppe oder Quotientengruppe

  * Der Faktorraum ist die Menge all dieser Äquivalenzklassen (Nebenklassen). Siehe auch: 
   * [Quotient Set](https://ncatlab.org/nlab/show/quotient+set) bzw. [Quotientenmenge](https://de.wikipedia.org/wiki/Äquivalenzrelation#Quotientenmenge_und_Partition)
   * [Quotient Space](https://en.m.wikipedia.org/wiki/Quotient_space_(topology) bzw. [Faktorraum](https://de.wikipedia.org/wiki/Faktorraum)
   * Quotient Topology bzw. ([Quotiententopologie](https://de.wikipedia.org/wiki/Quotiententopologie) 
   * [Quotient Category](https://en.m.wikipedia.org/wiki/Quotient_category)

**Mathematische Definition der Nebenklasse**

Definiert man auf der Gruppe $(G, *)$ mit einer Untergruppe $(H, *)$ die Relation $\sim$ durch

$
b \sim a: \Longleftrightarrow \exists h \in H: b=a * h
$

erhält man eine Äquivalenzrelation auf $G,$ die die Menge $G$ partitioniert. Die Äquivalenzklasse (englisch: coset) zu einem Element $a \in G$ (d. h. die Menge aller Elemente $b$, die zu $a$ in der Relation $b \sim a$ stehen), ist die Menge

$
\{b \in G \mid b \sim a\}=\{a * h \mid h \in H\}
$

Für diese Menge schreibt man $a * H, a H$ oder auch kurz $\bar{a},$ wenn klar ist, durch welchen Vorgang die Nebenklassen gebildet werden. Da diese Menge alle Elemente von $G$ enthäit, die dadurch entstehen, dass die Elemente aus $H$ links mit dem Element $a$ verknüpft werden, heißt sie (genauer) die Linksnebenklasse, Alternativbezeichnung Linksrest klasse, von $H$ nach dem Element $a$.

Wenn man andererseits eine Relation $\sim$ durch

$
b \sim a: \Longleftrightarrow \exists h \in H: b=h * a
$

definiert, dann ist dies im Allgemeinen eine andere Äquivalenzrelation und die Menge der zu a àquivalenten Elemente in $G$ jetzt

$
H a:=\{h * a \mid h \in H\}
$

die durch Rechtsverknüpfung der Elemente aus $H$ mit dem Element $a$ entsteht. Sie wird mit $H * a$ oder $H a$ bezeichnet und Rechtsnebenklasse, Alternativbezeichnung Rechtsrestklasse, von $H$ nach dem Element $a$
genannt.

Für die Menge aller Links- bzw. Rechtsnebenklassen wird die Notation

$
G / H:=\{g H \mid g \in G\}
$

bzw.

$
H \backslash G:=\{H g \mid g \in G\}
$

verwendet.

**Rechenbeispiel**

Dies sind alle **[Nebenklassen](https://de.wikipedia.org/wiki/Gruppentheorie#Nebenklassen)** von $6 \mathbb{Z},$ wie man leicht sehen kann, da sie die Gruppe $\mathbb{Z}$ partitionieren und:

$6 \mathbb{Z}+6=6 \mathbb{Z}+0$,

$6 \mathbb{Z}+7=6 \mathbb{Z}+1$,

$6 \mathbb{Z}+8=6 \mathbb{Z}+2$

und so weiter. 

Da die Operation in $\mathbb{Z}$ die Addition ist, nennt man die Addition der Nebenklassen auch Addition und es gilt beispielsweise $(6 \mathbb{Z}+3)+(6 \mathbb{Z}+4)=6 \mathbb{Z}+7=6 \mathbb{Z}+1 .$

1. Normalteiler N ist eine Untergruppe von G (N, $\circ$ ) $\subseteq$ (G, $\circ$ ) (die Verknüpfung ist die gleiche)

2. gN = Ng $\forall$ g $\in$ G


Beispiel:

N := (5 * $Z$, +) und G := ($Z$, +)

g = 2

dann: 

**Linksnebenklasse**: 2 * N := {..., 2+0, 2+5, 2+10,...)

**Rechtsnebenklasse**: N * 2 := {..., 0+2, 5+2, 10+2,..}

**In einer kommutativen Gruppe sind alle Untergruppen automatisch ein Normalteiler** (wenn ich aber eine andere Gruppe als $Z$ wähle, kann es sein, dass es nicht kommutativ ist.)

Beispiel ohne kommutative Gruppe: 

N = Alt (3) - alternierende Gruppe 3 $\subseteq$ Sym(3) - symmetrische Gruppe = {id, (12), (13), (23), (123), (132)}

![alternativer Text](https://raw.githubusercontent.com/deltorobarba/repo/master/integersmod5b.png)

![alternativer Text](https://raw.githubusercontent.com/deltorobarba/repo/master/integersmod5c.png)

#### **Homomorphiesatz**

**Homomorphiesatz (Fundamental Theorem on Homomorphism)**: Ist $f:(G, \circ) \rightarrow(H, \star)$ ein Gruppenhomomorphismus, dann ist der Kern $N:=\operatorname{ker}(f)$ ein Normalteiler von $G$ und die Faktorgruppe $G / N$ ist isomorph zum Bild $f(G)$. Ein entsprechender isomorphismus ist gegeben durch $\tilde{f}: G / N \rightarrow f(G) ; g N \mapsto f(g)$. [Source](https://de.wikipedia.org/wiki/Homomorphiesatz).

**The image of a homomorphism is isomorphic to the quotient by the kernel.** [Source](https://en.m.wikipedia.org/wiki/Kernel_(algebra))

**Normalteiler sind Kerne eines Gruppenhomorphismus**



![cc](https://raw.githubusercontent.com/deltorobarba/repo/master/homomorphy.jpg)

**Faktorgruppe nach Kernen von Homomorphismen**

![Image](https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Fundamental_Homomorphism_Theorem.svg/210px-Fundamental_Homomorphism_Theorem.svg.png)

If K = ker(f) we get the [first isomorphism theorem](https://en.m.wikipedia.org/wiki/Isomorphism_theorems) = der Normalteiler K ist der Kern ker(f) des Gruppenhomomorphismus. "**Every homomorphic image of a group is isomorphic to a quotient group**". [Source](https://en.m.wikipedia.org/wiki/Fundamental_theorem_on_homomorphisms)

**Bedingungen:**

* Let $G$ and $H$ be two groups.
* and let $f$ : $G \rightarrow H$ be a group homomorphism ([Gruppenhomomorphismus](https://de.wikipedia.org/wiki/Gruppenhomomorphismus)).
* and let $K$ be a normal subgroup (Normalteiler) in $G$ and $\varphi$ the natural surjective homomorphism $G \rightarrow G / K$ (where $G / K$ is a quotient group). Diese Faktorgruppen sind homomorphe Bilder von G und **jedes homomorphe Bild von G ist zu einer solchen Faktorgruppe G/K isomorph**.

**Then:**

1. **Dann ist der Kern von $f$ ein Normalteiler von $G$.**
  * Normalteiler sind die [Kerne](https://de.m.wikipedia.org/wiki/Kern_(Algebra)) von Gruppenhomomorphismen, weshalb dann klar ist, dass umgekehrt der Kern von $f$ ein Normalteiler von $G$ ist.

  * If $K$ is a **subset** of ker $(f)$ then there exists a unique homomorphism $h: G / K \rightarrow H$ such that $f=h$ $\varphi$. In other words, the natural projection $\varphi$ is universal among homomorphisms on $G$ that map $K$ to the identity element.

2. **und daher kann die Faktorgruppe $G /$ ker $f$ gebildet werden.** 

3. **Nach dem [Homomorphiesatz](https://de.wikipedia.org/wiki/Homomorphiesatz) ist diese Faktorgruppe $G /$ ker $f$ isomorph zum Bild von $f$, das eine Untergruppe von $H$ ist.** 
  * The image of $f$ is isomorphic to the quotient group $G /$ ker ($f$). And in particular, if $f$ is surjective then $H$ is isomorphic to $G$ / ker $(f)$. [Source](https://en.m.wikipedia.org/wiki/Isomorphism_theorems#First_Isomorphism_Theorem_4)
  * The image of $f$ is hierbei a subgroup of $H$.

![cc](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7d/Group_homomorphism_ver.2.svg/500px-Group_homomorphism_ver.2.svg.png)

*Image of a group homomorphism (h) from G (left) to H (right).*

*The smaller oval inside H is the image of h. N is the kernel of h and aN is a coset of N.* [Source](https://en.m.wikipedia.org/wiki/Group_homomorphism)

See also: https://mathepedia.de/Kern_und_Bild_Homomorphismus.html

**Homomorphiesatz (allgemein)**: 

* Aus einer Abbildung $f$ zwischen zwei Gruppen $G$ und $H$, die weder injektiv noch surjektiv ist, wollen wir eine bijektive Abbildung herleiten.

* Schritt 1: Das macht man, indem man zuerst auf der rechten Seite alle Elemente ausschliesst, die nicht Teil der Zielmenge sind, und sich nur auf die 'getroffenen' Elemente fokussiert (The image of $f$ is hierbei a subgroup of $H$.)

* Schritt 2: Jetzt hat man nur noch das Problem auf der linken Seite, dass mehrere Startelemente in $G$ auf ein und dasselbe Zielelement in $H$ verweisen. Man betrachtet die Startelemente dann einfach als identisch. Das macht man in dem man $G$ umwandelt in $G$ / Kern ($f$) (man bildet die Faktorgruppe, und spricht aus: modulo Kern von f. Der Quotientenvektorraum von $G$ nach kern von $f$. 
  * Das geht, weil $f$ ein Homomorphismus ist
  * Das bedeutet, wenn zwei Elemente a und b im Start auf ein Element im Ziel verweisen, dann unterscheiden sie sich um ein Kernelement. Heisst, a minus b ist ein Element, das auf 0 geschickt wird, und damit ein Kernelement.
  * Und wenn ich jetzt modulo des kerns rechne, dann tue ich so, als ob es die Differenz nicht gibt. Weil es bedeutet a =b. Der Quotientenraum ist ein kunstlich geschaffener Raum, wo a und b identisch gemacht wurden (kongruent).
  * Remember aus Modulorechnung: Zwei Zahlen sind kongruent (modulo des Moduls m), wenn ihre Differenz durch m teilbar ist. Hier ist das Modulo der Kern. Also Rest muss 0 sein.
  * Modulo n (Reste berechnen, hierbei 0): die Differenz zweier Elemente ist teilbar durch n. Die beiden Elemente sind dann kongruent (identisch).
  * $G$ modulo Kern $f$ ist isomorph (identisch =bijektiv und homomorph)) zu Bild $f$, das in $H$ liegt.

Jäger & Hasen: https://youtu.be/QA9rrDMlaHc

**Important basis: What is a Kernel of a Group homomorphism?**: 

* A homomorphism between two groups does not have to be a one-to-one function. 

* If it's not a one-to-one function, then **there's a group associated with the homomorphism which measures the degree to which the function is not an injection. This group is called the kernel**. (See video [The Kernel of a Group Homomorphism](https://www.youtube.com/watch?v=TngePpJ_x-I&list=WL&index=23))

* Homomorphisms send identity elements from one group to identity elements of another, and same with inverses (a 1 - 1 relationship)

* Now suppose f: $G$ -> $H$ is not 1-1. Then there are at least two elements in G which map to the same element in H, like: x1, x1, .. $\mapsto$ y

> Suppose $f: G \rightarrow H$ is not $1-1$
$x_{1}, x_{2}, \ldots \rightarrow y$

* This means f(x1) = y and f(x2) = y as well. Now watch what happens if we multiply all of these equalities by f(x1) inverses. 

> $f\left(x_{1}\right)=y \Rightarrow f\left(x_{1}\right) \diamond f\left(x_{1}^{-1}\right)=y \diamond f\left(x_{1}^{-1}\right)$

> $f\left(x_{2}\right)=y \Rightarrow f\left(x_{2}\right) \diamond f\left(x_{1}^{-1}\right)=y^{\wedge} f\left(x_{1}^{-1}\right)$

* Since homomorphisms send inverses to inverses, f(x1) inverse equals y inverse. This allows us to simplify the right hand sides:

> $f\left(x_{1}\right)=y \Rightarrow f\left(x_{1}\right) \diamond f\left(x_{1}^{-1}\right)=y^{\diamond} y^{-1}$

> $f\left(x_{2}\right)=y \Rightarrow f\left(x_{2}\right) \diamond f\left(x_{1}^{-1}\right)=y^{\diamond} y^{-1}$

* Next we can combine the left hand sides, because F is a homomorphism.

> $f\left(x_{1}\right)=y \Rightarrow f\left(x_{1}\right) \diamond f\left(x_{1}^{-1}\right)$ = 1 $_{H}$

> $f\left(x_{2}\right)=y \Rightarrow f\left(x_{2}\right) \diamond f\left(x_{1}^{-1}\right)$ = 1 $_{H}$

* This shows that there are multiple elements in G which all map to the identity in H. These elements are called the Kernel of f:

> $\operatorname{ker}(f)=\left\{x \in G \mid f(x)=1_{H}\right\}$

*Diagram of the fundamental theorem on homomorphisms where f is a homomorphism, N is a normal subgroup of G and e is the identity element of G.*

![Image](https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/Diagram_of_the_fundamental_theorem_on_homomorphisms.svg/440px-Diagram_of_the_fundamental_theorem_on_homomorphisms.svg.png)

![Image](https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Fundamental_Homomorphism_Theorem.svg/210px-Fundamental_Homomorphism_Theorem.svg.png)

The situation is described by this commutative diagram. By setting K = ker(f) we immediately get the [first isomorphism theorem](https://en.m.wikipedia.org/wiki/Isomorphism_theorems). We can write the statement of the fundamental theorem on homomorphisms of groups as "**Every homomorphic image of a group is isomorphic to a quotient group**".

[Source](https://en.m.wikipedia.org/wiki/Fundamental_theorem_on_homomorphisms)

Sources: [Fundamental theorem on homomorphisms](https://en.m.wikipedia.org/wiki/Fundamental_theorem_on_homomorphisms) & [Homomorphiesatz](https://de.wikipedia.org/wiki/Homomorphiesatz) & [Quotient (universal algebra)](https://en.m.wikipedia.org/wiki/Quotient_(universal_algebra)) & [Exact sequence](https://en.m.wikipedia.org/wiki/Exact_sequence) & [Isomorphism theorems](https://en.m.wikipedia.org/wiki/Isomorphism_theorems#First_Isomorphism_Theorem_4)

#### **Exact Sequences**



In the context of group theory, a sequence

$
G_{0} \stackrel{f_{1}}{\longrightarrow} G_{1} \stackrel{f_{2}}{\longrightarrow} G_{2} \stackrel{f_{3}}{\longrightarrow} \cdots \stackrel{f_{n}}{\longrightarrow} G_{n}
$

of groups and group homomorphisms is called exact if the image of each homomorphism is equal to the kernel of the next:

$
\operatorname{im}\left(f_{k}\right)=\operatorname{ker}\left(f_{k+1}\right)
$

The sequence of groups and homomorphisms may be either finite or infinite.

![ccc](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Illustration_of_an_Exact_Sequence_of_Groups.svg/1066px-Illustration_of_an_Exact_Sequence_of_Groups.svg.png)

Img (g) = Ker (f)

https://en.wikipedia.org/wiki/Exact_sequence

Illustration of an exact sequence of groups $G_{i}$ using Venn diagrams. Each group homomorphism $f_{i}: G_{i-1} \rightarrow G_{i}$ maps $G_{i-1}$ to the kernel of the next homomorphism. This is depicted by the reduction of the subgroups from left to right. [Source](https://en.m.wikipedia.org/wiki/Exact_sequence)

https://youtu.be/rXEiJhBHJsU

https://youtu.be/AZnJFyDOq3E

https://youtu.be/kCPYALlpECk

#### **Simplicial Homology**

##### **Overview**

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_04.png)

Vergleich mit paper von Krishna 2017

5 kongruent 11 modulo 3, weil 3 * 1 + 2 = 5, und 3 * 3 + 2 + 11, Reste müssen identisch sein, sowie 17 etc.

- drei Basiselemente: 0, 1 und 2
- 3 ist wie Bp, also Image (element des vektorraums, und ist linear, weil zB multiplizier mit Skala bleibt im vektorraum)
- Rest ware Hp, also 2 (das Loch)
- 5 und 11 und 17 sind Zp (Kern)
- Berate alle Objekte aus der Filtration
- Ein element aus Zp was plus ein Element aus Bp

[Vergleich mit paper von Krishna 2017]

5 kongruent 11 und 17 etc. modulo 3, weil (3 * 1) + 2 = 5, und (3 * 3) + 2 = 11. Reste müssen identisch sein.
* **Zp (Kern)**: 5, 11, 17 etc. (alle Objekte aus einer Filtration zB in Cp, die kongruent zueinander sind, **weil deren Differenz ein ganzzahliges Vielfaches von Bp (modulo) ist)**.
* **Hp (Hom)**: 2 (= das Loch)
* **Bp (Img)**: 3 (modulo)
- drei Basiselemente: 0, 1 und 2 (?)

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_01.png)

* Zp (Ker): 5, 11, 17 etc. (alle Objekte aus einer Filtration zB in Cp, die kongruent zueinander sind, weil deren Differenz ein ganzzahliges Vielfaches von Bp (modulo) ist).
* Hp (Hom): 2 (= das Loch)
* Bp (Img): 3 (modulo)

**Vergleich mit exact sequence: im⁡( f k ) = ker( f k + 1 ) bzw. Zp = Bp mit Hp = 0**

![cc](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Illustration_of_an_Exact_Sequence_of_Groups.svg/640px-Illustration_of_an_Exact_Sequence_of_Groups.svg.png)

* **Äquivalenzklassen**: überflüssige Elemente entfernen, die sonst alle zum gleichen Ergebnis führen

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_02.png)

Boundary von boundary is Null wegen Orienting: vertices gleiche sich aus (Orientierung)

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_03.png)

**Homology**
- Simplex: https://en.wikipedia.org/wiki/Simplex 
- Simplicial Complex: https://en.wikipedia.org/wiki/Simplicial_complex 
- Chain Complex: https://en.wikipedia.org/wiki/Chain_complex
- Homology: Associated to a chain complex is its homology, which describes how the images are included in the kernels.
- Homological Algebra: https://en.wikipedia.org/wiki/Homological_algebra 
- Boundary operator = boundary homomorphism

Since Bn is a subgroup of Zn, we may form the quotient group Hn = Zn/Bn -> so Modulo (=Restwerte) ist dann die Dimension der Löcher (=invarianten)?

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_06.png)

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_05.png)

https://youtu.be/QA9rrDMlaHc (Homomorphiesatz)

https://youtu.be/390eRzVSC2k (Homomorphie mit Modulo und kommutativen Diagramm)

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_07.png)

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_10.png)

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_08.png)

![bb](https://raw.githubusercontent.com/deltorobarba/repo/master/simplicial_homology_09.png)

**Simplicial Homology Groups**

**Homology**

* Homology counts components, holds, voids etc.

* In topology we define something called homology for simplicial complexes.

* Homology of a simplicial complex is computable via linear algebra.

* Die Homologie ist ein mathematischer Ansatz, die Existenz von Löchern zu formalisieren.

* Gewisse „sehr feine“ Löcher sind für die Homologie unsichtbar; hier kann u. U. auf die schwerer zu bestimmenden Homotopiegruppen zurückgegriffen werden.

**Simplicial Homology**

* Die Simpliziale Homologie ist in der Algebraischen Topologie, einem Teilgebiet der Mathematik, eine Methode, die einem beliebigen Simplizialkomplex **eine Folge [abelscher Gruppen](https://de.m.wikipedia.org/wiki/Abelsche_Gruppe) zuordnet**. 

* Anschaulich gesprochen zählt sie die Löcher unterschiedlicher Dimension des zugrunde liegenden Raumes.

https://de.m.wikipedia.org/wiki/Simpliziale_Homologie

https://en.m.wikipedia.org/wiki/Simplicial_homology

Homology is a general way of associating a sequence of algebraic objects, such as abelian groups or modules, to other mathematical objects such as topological spaces.

In homology we want to Linearize the equivalence relation! Homology and cohomology are linear theories (easier to compute, and methods of linear algebra applicable) (though in the process you loose a bit information

1. define set of vertices V0 (=vector space V0 with basis given by the set of vertices) and set of edges V1

2. linearize equivalence relation: instead of looking at „tail of edge is equivalent to head of edge“, we consider the difference between these two
    * **e $\mapsto$ h(e) - t(e)** is the linear combination of two vertices, and hence an element of e zero! e wird abgebildet auf dem Element (h(e) - t(e))
    * **h(e) $\equiv$ t(e) mod (im d)** das heisst: h(e) ist identisch zu t(e) modulo dem Bild von d

* The linear map d : V1 -> V0 sends an edge e to (h(e) - t(e))
* e is an edge which is a basis element inside V1
* So the image of d (difference) is just the supspace of V0 generated by these elements
* Generate equivalence relation: impose reflexivity, symmetry and transivity. 
* So: to get the image of d you look at the subspace generated by e |—> h(e) - t(e)
    * Closure under addition more or less corresponds to transitivity, 
    * closure under negation corresponds to symmetry, and 
    * closure under zero corresponds to reflexivity. 
    * So this precisely linearizes the equivalence relation from before. 
* Now we have one linear map. But we need two linear maps which composes 0 to get homology or cohomology. There are two ways you can get a composite to get 0 very easily. 
    * Either start from zero and map to V1 and then go to V0, or 
    * start at V1 and use d to get to V0 and then go to 0 with the zero map there.
* Modular the image of this map which is zero

https://ncatlab.org/nlab/show/homology

https://en.m.wikipedia.org/wiki/Homology_(mathematics)

**Homology Groups**

* Homology is a general way of associating a sequence of algebraic objects, such as abelian groups or modules, to other mathematical objects such as topological spaces.

* In homology we want to Linearize the equivalence relation! Homology and cohomology are linear theories (easier to compute, and methods of linear algebra applicable) (though in the process you loose a bit information

**Cycles (and its Kern)**

A cycle is a closed submanifold, a boundary is a cycle which is also the boundary of a submanifold.

**Boundary operator on chains (and its image)**

The boundary of a chain is the linear combination of boundaries of the simplices in the chain. The boundary of a k-chain is a (k−1)-chain. Note that the boundary of a simplex is not a simplex, but a chain with coefficients 1 or −1 – thus chains are the closure of simplices under the boundary operator.

![cc](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7f/Chainline.svg/320px-Chainline.svg.png)

*The boundary of a polygonal curve is a linear combination of its nodes; in this case, some linear combination of A1 through A6. Assuming the segments all are oriented left-to-right (in increasing order from Ak to Ak+1), the boundary is A6 − A1.*

If the 1 -chain $c=t_{1}+t_{2}+t_{3}$ is a path from point $v_{1}$ to point $v_{4},$ where $t_{1}=\left[v_{1}, v_{2}\right], t_{2}=\left[v_{2}, v_{3}\right]$ and $t_{3}=\left[v_{3}, v_{4}\right]$ are its
constituent 1 -simplices, then
$$
\begin{aligned}
\partial_{1} c &=\partial_{1}\left(t_{1}+t_{2}+t_{3}\right) \\
&=\partial_{1}\left(t_{1}\right)+\partial_{1}\left(t_{2}\right)+\partial_{1}\left(t_{3}\right) \\
&=\partial_{1}\left(\left[v_{1}, v_{2}\right]\right)+\partial_{1}\left(\left[v_{2}, v_{3}\right]\right)+\partial_{1}\left(\left[v_{3}, v_{4}\right]\right) \\
&=\left(\left[v_{2}\right]-\left[v_{1}\right]\right)+\left(\left[v_{3}\right]-\left[v_{2}\right]\right)+\left(\left[v_{4}\right]-\left[v_{3}\right]\right) \\
&=\left[v_{4}\right]-\left[v_{1}\right]
\end{aligned}
$$

![cc](https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/Closed_polygonal_line.svg/320px-Closed_polygonal_line.svg.png)

*A closed polygonal curve, assuming consistent orientation, has null boundary. (deswegen führt der boundary operator δ1 alle Werte immer in Null, zumindest bei geschlossenen objekten)*

* Example 2: The boundary of the triangle is a formal sum of its edges with signs arranged to make the traversal of the boundary counterclockwise.

   * Cycle: **A chain is called a cycle when its boundary is zero**. 
   
   * Boundary: A chain that is the boundary of another chain is called a boundary. 

   * **Boundaries are cycles**, so chains form a chain complex, whose homology groups (cycles modulo boundaries) are called simplicial homology groups.

* Example 3: A 0-cycle is a linear combination of points such that the sum of all the coefficients is 0. Thus, the 0-homology group measures the number of path connected components of the space.

* Example 4: The plane punctured at the origin has nontrivial 1-homology group **since the unit circle is a cycle, but not a boundary.**

https://m0nads.wordpress.com/tag/persistent-homology/

https://en.m.wikipedia.org/wiki/Chain_(algebraic_topology)

**Homology Group & Betti Number**

Let $\sigma=\left(v_{0}, \ldots, v_{k}\right)$ be an oriented $k$ -simplex, viewed as a basis element of $C_{k}$. The boundary operator

$
\partial_{k}: C_{k} \rightarrow C_{k-1}
$

is the homomorphism defined by:

$
\partial_{k}(\sigma)=\sum_{i=0}^{k}(-1)^{i}\left(v_{0}, \ldots, \widehat{v_{i}}, \ldots, v_{k}\right)
$

where the oriented simplex

$
\left(v_{0}, \ldots, \widehat{v_{i}}, \ldots, v_{k}\right)
$

is the $I^{\text {th }}$ face of $\sigma,$ obtained by deleting its $i^{\text {th }}$ vertex.
In $C_{k},$ elements of the subgroup

$
Z_{k}:=\operatorname{ker} \partial_{k}
$

are referred to as cycles, and the subgroup

$
B_{k}:=\operatorname{im} \partial_{k+1}
$

is said to consist of boundaries.

The $k^{\text {th }}$ homology group $H_{k}$ of $S$ is defined to be the [quotient abelian group](https://en.m.wikipedia.org/wiki/Quotient_group)

$
H_{k}(S)=Z_{k} / B_{k}
$

* It follows that the **homology group $H_{k}(S)$ is nonzero exactly when there are $k-$
cycles on $S$ which are not boundaries**. In a sense, this means that there are $k$ -
dimensional holes in the complex. 

* For example, consider the complex $S$ obtained by gluing two triangles (with no interior) along one edge, shown in the image. The edges of each triangle can be oriented so as to form a cycle. These two cycles are by construction not boundaries (since every 2 -chain is zero). 

* One can compute that the homology group $\mathrm{H}_{1}(\mathrm{S})$ is isomorphic to $\mathrm{Z}^{2}$, with a basis given by the two cycles mentioned. This makes precise the informal idea that $S$ has two "1-
dimensional holes".

* Holes can be of different dimensions. The rank of the $k$ th homology group, the
number

$
\beta_{k}=\operatorname{rank}\left(H_{k}(S)\right)
$

is called the $k$ th Betti number of $S$. It gives a measure of the number of $k$ -
dimensional holes in $S$.

**A homology class is thus represented by a cycle which is not the boundary of any submanifold: the cycle represents a hole, namely a hypothetical manifold whose boundary would be that cycle, but which is "not there".**

* An homology class Hn (which represents a hole) is an [equivalence class](https://en.m.wikipedia.org/wiki/Equivalence_class) of 
    * [cycles](https://en.m.wikipedia.org/wiki/Simplicial_homology#Boundaries_and_cycles) Ker(δ) modulo boundaries Im(δ) bzw. 
    * h(e) $\equiv$ t(e) mod (im d)

* Ker(δ) kann beschrieben werden: h(e) $\equiv$ t(e) bzw. e $\mapsto$ h(e) - t(e) or: ∂n−1 ◦ ∂n = 0

* This is the linear combination of two vertices, and hence an element of e zero

* Get the image of d bzw. Im(δ) when you look at the subspace generated by e $\mapsto$ h(e) - t(e)

*  Im(δ): boundaries?

A sequence of groups that is not a chain complex: a map consists of multiplication by 2 in each step! This is not a chain complex, since 2·2=4, i.e., ∂n−1 ◦∂n = 4 = not 0.
* So does this mean that Ker(δ) bzw. h(e) =congruent= t(e) bzw. e|—> h(e) - t(e) beschreibt einen Chain Complex?
* und in Verbindung mit Im(δ) (boundaries?) beschreibt es eine Homology?

https://en.m.wikipedia.org/wiki/Simplicial_homology#Boundaries_and_cycles

https://ncatlab.org/nlab/show/boundary+of+a+simplex

https://ncatlab.org/nlab/show/chain+map

https://en.m.wikipedia.org/wiki/Spectral_sequence

##### **Homological Algebra**

* Why using Homological Algebra: **translate a problem of interest into a sequence of „higher“ (=homological algebra) linear algebra problems**

* homological algebra is the **study of homological functors** and the intricate algebraic structures that they entail. 

* One quite useful and ubiquitous concept in mathematics is that of **chain complexes**, which can be studied through both their homology and cohomology.

* Homological algebra affords the means to **extract information contained in these complexes and present it in the form of homological invariants of rings, modules, topological spaces**, and other 'tangible' mathematical objects. A powerful tool for doing this is provided by spectral sequences.

https://en.m.wikipedia.org/wiki/Homological_algebra

##### **Simplicial Complex & Triangulation**

![vvv](https://raw.githubusercontent.com/deltorobarba/repo/master/homology_03.jpg)

**Simplices**

A simplex consists of 3 components: vertices, edges and faces

* 0-simplex: point
* 1-simplex: edge (line)
* 2-simplex: 3 connected points
* 3-simplex: solid (3 dimensions)

k-simplex is k-dimensional is formed using (k+1) vertices ('convex hull')

* Complete graphs (every vertice is connected to all others) can be interpreted as simplices

* you can interpret any graph as simplicial complex

https://de.m.wikipedia.org/wiki/Simplex_(Mathematik)

**Simplicial Complexes**

* Mit einem Simplizialkomplex können die entscheidenden Eigenschaften von triangulierbar topologischen Räumen algebraisch charakterisiert werden können

* Warum? Definition von Invarianten im topologischen Raum. Simplicial complexes can be seen as higher dimensional generalizations of neighboring graphs.

* Wie? Untersuchung eines topologischen Raums durch Zusammenfügen von Simplizes womit eine Menge im d-dimensionalen euklidischen Raum konstruiert wird, die [homöomorph](https://de.m.wikipedia.org/wiki/Homöomorphismus) ist zum gegebenen topologischen Raum. 

* Die „Anleitung zum Zusammenbau“ der Simplizes, das heißt die Angaben darüber, wie die Simplizes zusammengefügt sind, wird dann in Form einer Sequenz von [Gruppenhomomorphismen](https://de.m.wikipedia.org/wiki/Gruppenhomomorphismus) rein algebraisch charakterisiert.

* Cells can have various dimensions: vertices, edges, triangles, tetrahedra and their higher dimensional analogues. complexes reflect the correct topology of the data

* If we glue many simplices together in such a way that the intersection is also a simplex (along an edge for example), we obtain a simplicial complex. If we see three points connected by edges that form a triangle, we fill in the triangle with a 2-dimensional face. Any four points that are all pairwise connected get filled in with a 3-simplex etc. The resulting simplicial complex is called (Vietoris) Rips complex.

* The persistence diagram is not computed directly from L􏰑ε. Instead, one forms an object called a Cech complex. Simplicial and cubical complexes are examples of cell complexes. Cech complex is an example of a simplicial complex. - in practice, often used Vietoris-Rips complex Vε - the persistent homology defined by Vε approximates the persistent homology defined by Cε.


**Čech complex**

* Subform of (Abstract) Cell complex: Simplicial Complex: Vietoris Rips Complex: Čech complex

* Other Simplicial Complexes besides Vietoris Rips Complex are witness complex, Clique complexes, flag complexes, and conformal hypergraphs.

* Other abstract cell complexes besides Simplicial Complexes is the Cubical Complex

* Čech complex is an abstract simplicial complex constructed from a point cloud in any metric space which is meant to capture topological information about the point cloud or the distribution it is drawn from. Given a finite point cloud X and an ε > 0, we construct the Čech complex C if the set of ε-balls centered at points of σ has a nonempty intersection. In other words, the Čech complex is the nerve of the set of ε-balls centered at points of X. By the nerve lemma, the Čech complex is homotopy equivalent to the union of the balls.

**Abstract Simplicial Complex**

https://en.m.wikipedia.org/wiki/Abstract_simplicial_complex

**n-skeleton**

![vvv](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Hypercubestar.svg/240px-Hypercubestar.svg.png)

https://en.m.wikipedia.org/wiki/N-skeleton

**Triangulation**

* In der Topologie ist eine Triangulierung oder Triangulation eine **Zerlegung eines Raumes in Simplizes** (Dreiecke, Tetraeder oder deren höher-dimensionale Verallgemeinerungen).

* Mannigfaltigkeiten bis zur dritten Dimension sind stets triangulierbar. 

* Ursprüngliche Motivation für die Hauptvermutung war der Beweis der topologischen Invarianz kombinatorisch definierter Invarianten wie der [simplizialen Homologie](https://de.m.wikipedia.org/wiki/Simpliziale_Homologie). Trotz des Scheiterns der Hauptvermutung lassen sich Fragen dieser Art oftmals mit dem [simplizialen Approximationssatz](https://de.m.wikipedia.org/wiki/Simpliziale_Approximation) beantworten.

* Triangulation ist eine Zerlegung eines Raumes in Simplizes (Dreiecke, Tetraeder oder höher-dimensionale Verallgemeinerungen) to tease out properties of manifolds

* **A triangulation of a topological space X is a simplicial complex K, homeomorphic to X, together with a homeomorphism h: K → X**

* triangulation offers a concrete way of visualizing spaces that are difficult to see, and helps computing an invariant.

* Ist gegeben durch einen (abstrakten) Simplizialkomplex K und Homöomorphismus h : | K | → X der geometrischen Realisierung | K | auf X. 
https://en.wikipedia.org/wiki/Triangulation_(topology) 

* a two-dimensional sphere (surface of a solid ball) can be approximated by gluing together two-dimensional triangles, and a three-dimensional sphere can be approximated by gluing together three-dimensional tetrahedra. 

* Triangles and tetrahedra are examples of more general shapes called simplices, which can be defined in any dimension.




https://en.m.wikipedia.org/wiki/Triangulation_(topology)

https://de.m.wikipedia.org/wiki/Triangulierung_(Topologie)

##### **Chain Complex**

Chain Complex is a sequence of homomorphism of abelian groups

A chain complex $V$. is a sequence $\left\{V_{n}\right\}_{n \in \mathbb{Z}}$ of abelian groups or modules (for instance yector spaces) or similar equipped with linear maps $\left\{d_{n}: V_{n+1} \rightarrow V_{n}\right\}$ such that $d^{2}=0,$ i.e. the composite of two consecutive such maps is the zero morphism $d_{n} \circ d_{n+1}=0$

https://ncatlab.org/nlab/show/chain+complex

https://ncatlab.org/nlab/show/zero+morphism

https://m0nads.wordpress.com/tag/persistent-homology/

![vv](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Simplicial_homology_-_exactness_of_boundary_maps.svg/384px-Simplicial_homology_-_exactness_of_boundary_maps.svg.png)

* The boundary of a boundary of a 2-simplex (left) and the boundary of a 1-chain (right) are taken. 

* Both are 0, being sums in which both the positive and negative of a 0-simplex occur once. The boundary of a boundary is always 0. 

* A nontrivial cycle is something that closes up like the boundary of a simplex, in that its boundary sums to 0, but which isn't actually the boundary of a simplex or chain. 

* Because trivial 1-cycles are equivalent to 0 in H1, the 1-cycle at right-middle is homologous to its sum with the boundary of the 2-simplex at left.

https://en.m.wikipedia.org/wiki/Chain_complex

https://www.math3ma.com/blog/open-sets-are-everything

https://www.math.uni-bielefeld.de/~sek/top/leit/leit8.htm

https://www.math.uni-bielefeld.de/~sek/top/leit/alg2.htm

https://github.com/GUDHI/TDA-tutorial/blob/master/Tuto-GUDHI-optimization.ipynb

https://tda-in-ml.github.io

https://bastian.rieck.me/blog/posts/2020/topology_graphs/

https://m0nads.wordpress.com/tag/persistent-homology/

https://colab.research.google.com/github/lewtun/hepml/blob/master/notebooks/lesson06_persistent-homology.ipynb

https://colab.research.google.com/github/lewtun/hepml/blob/master/notebooks/lesson07_topological-time-series.ipynb

##### **Topological Data Analysis**

![xxx](https://raw.githubusercontent.com/deltorobarba/repo/master/tda_01.jpg)

**Connected Components**

How the graph can be used to capture connected component

* Any graph gives us a topological space (like joining a square as a connected component)
* Graph can capture connected components via putting an equivalence relation on the set of vertices.
* equivalence relation is genersted by tail of an edge (vertex 1) is equivalent to a head of an edge (vertex 2) / between the vertices of an edge (which together forms a connected component). 
* To generate equivalence relation we have to make sure that we enforce reflexivity, symmetry and transivity.
* You get equivalence classes

https://en.m.wikipedia.org/wiki/Component_(graph_theory)

https://en.m.wikipedia.org/wiki/Connected_space

**Filtration & Inclusion Maps**

![ccc](https://raw.githubusercontent.com/deltorobarba/repo/master/homology_01.jpg)

* With increasing size of d, we are dealing with a sequence of simplicial complexes, each a sub-complex of the next. That is, a simplicial complex constructed from data for some small distance is a subset of the simplicial complex constructed for a larger distance.

* Equally there is an **inclusion map from each simplicial complex to the next**. 

* **This sequence of simplicial complexes, with inclusion maps, is called a filtration**. 

* When we apply homology to a filtration, we obtain an algebraic structure called persistent modul.

* So if we want to compute **ith homology with coefficients from a field k**. 

* The **homology of any complex Cj is a vector space**, and the **inclusion maps between complexes induce linear maps between homology vector spaces**. 

* The direct sum of the homology vector spaces is an algebraic module - in fact a **graded module over the polynomial ring** k[x]. The variable x acts as a **shift map**, taking each homology generator to its image in the next vector space.

* Furthermore, a structure theorem tells us that **a persistent module decomposes nicely into a direct sum of simple modules**, each corresponding to a bar in the barcode. This means: a barcode reall is an algebraic structure.

![cscsvs](https://raw.githubusercontent.com/deltorobarba/repo/master/homology_02.jpg)

**Persistent Homology**

* Persistent Homology is an algebraic method for discerning topological features of data

* Topological features can be: components, clusters, holes, graph structures).

* Data can be: for example set of discrete points with a metric that gives distance between pairs of points.

![vvv](https://raw.githubusercontent.com/deltorobarba/repo/master/homology_04.jpg)

![vvv](https://raw.githubusercontent.com/deltorobarba/repo/master/homology_05.jpg)

1. Choose a distance d and draw a ball of diameter d around each point.

2. Simplices: Two balls intersect exactly when two points are no further apart than distance d, in which case we connect the two points with an edge. This creates a graph whose vertices are the original points. Problem: The graph shows us that the points form a single cluster at scale parameter d, but it doesn‘t tell us about higher-order feature, like holes. For example, this groph has many cycles, but the graph doesn‘t help us identify the central hole in data.

3. Fill in complete simplices to obtain the Vietoris Rips complex 

4. We then apply homology to this complex, which reveals the presence of holes.

**Persistent Barcode, Diagram & Landscape**

* What is the ideal size of d? - Consider all distances d between d1 min (to connect two balls) and d2 max size (all are connected). Each hole appears at a particular value of d and disappears at anotjer value of d. We can represent the persistence of this hole as a pair (d1, d2).
* Out of this distance we get a bar. Several holes result in a barcode. Short bars represent noise. Long bars are features.
* Persistent barcodes are stable with respect to pertubations if data (Edelsbrunner 2007).
* Barcode is computable via linear algebra. Runtime is O (n3), it‘s cubic, where n is the number of simplices (Carlsson 2005).
* A barcode is a visualization of an algebraic structure.

![vvv](https://raw.githubusercontent.com/deltorobarba/repo/master/homology_06.jpg)

![vvv](https://raw.githubusercontent.com/deltorobarba/repo/master/homology_07.jpg)

## **Point-Set Topology**

##### **Overview**

General topology is the branch of topology that deals with the basic set-theoretic definitions and constructions used in topology. It is the foundation of most other branches of topology, including differential topology, geometric topology, and algebraic topology. **Another name for general topology is point-set topology**.

The fundamental concepts in point-set topology are continuity, compactness, and connectedness:

* **Continuous functions**, intuitively, take nearby points to nearby points.

* **Compact sets** are those that can be covered by finitely many sets of arbitrarily small size.

* **Connected sets** are sets that cannot be divided into two pieces that are far apart.

**Once a choice of open sets is made, the properties of continuity, connectedness, and compactness, which use notions of nearness, can be defined using these open sets.**

* **Point-Set Topology**: The low-level language of topology, which is not really considered a separate "branch" of topology. 

* Point-set topology, also called set-theoretic topology or general topology, is the study of the general abstract nature of continuity or "closeness" on spaces. 

* Basic point-set topological notions are ones like **continuity, dimension, compactness, and connectedness**. The intermediate value theorem (which states that if a path in the real line connects two numbers, then it passes over every point between the two) is a basic topological result. Others are that Euclidean n-space is homeomorphic to Euclidean m-space iff m=n, and that real valued functions achieve maxima and minima on compact sets.

* Foundational point-set topological questions are ones like "when can a topology on a space be derived from a metric?" Point-set topology deals with differing notions of continuity and compares them, as well as dealing with their properties. 

* Point-set topology is also the ground-level of inquiry into the geometrical properties of spaces and continuous functions between them, and in that sense, it is the foundation on which the remainder of topology (algebraic, differential, and low-dimensional) stands.

https://mathworld.wolfram.com/Point-SetTopology.html

##### **Continuity**

* A continuous function is a function that does not have any abrupt changes in value, known as discontinuities. 

* More precisely, sufficiently small changes in the input of a continuous function result in arbitrarily small changes in its output.

https://en.m.wikipedia.org/wiki/Continuous_function

##### **Connectedness**

* In topology and related branches of mathematics, **a connected space is a topological space that cannot be represented as the union of two or more disjoint non-empty open subsets**. 

* Connectedness is one of the principal topological properties that are used to distinguish topological spaces.

* **A subset of a topological space X is a connected set if it is a connected space when viewed as a subspace of X.**

* A topological space X is said to be disconnected if it is the union of two disjoint non-empty open sets. Otherwise, X is said to be connected. 

* A subset of a topological space is said to be connected if it is connected under its subspace topology. Some authors exclude the empty set (with its unique topology) as a connected space, but this article does not follow that practice.

**Connected and disconnected subspaces of R²**

![xx](https://upload.wikimedia.org/wikipedia/commons/thumb/1/16/Simply_connected%2C_connected%2C_and_non-connected_spaces.svg/192px-Simply_connected%2C_connected%2C_and_non-connected_spaces.svg.png)

* **Connected**: red space A, pink space B, yellow space C and orange space D
* **Disconnected**: green space E (made of subsets E1, E2, E3, and E4) 
* **Simply connected**: A and B (genus 0), 
* **Not simply connected**: C and D: C has genus 1 and D has genus 4.

Genus (Geschlecht): Anzahl der Löcher in einer Fläche (https://de.m.wikipedia.org/wiki/Geschlecht_(Fläche))

https://en.m.wikipedia.org/wiki/Connected_space

##### **Compactness**

**// Open & Closed Sets (Abgeschlossen)**

* In mathematics, particularly in topology, an open set is an abstract concept generalizing the idea of an open interval in the real line.

* **An open interval does not include its endpoints, and is indicated with parentheses**. For example, (0,1) means greater than 0 and less than 1. This means (0,1) = {x | 0 < x < 1}. (*"(0,1) is the set of all numbers x, such that x is an integer in the range greater than 0 to 1 exclusive"*)

* A closed interval is an interval which includes all its limit points, and is denoted with square brackets. For example, [0,1] means greater than or equal to 0 and less than or equal to 1. 

* A half-open interval includes only one of its endpoints, and is denoted by mixing the notations for open and closed intervals. (0,1] means greater than 0 and less than or equal to 1, while [0,1) means greater than or equal to 0 and less than 1.

* Once a choice of open sets is made, the properties of continuity, connectedness, and compactness, which use notions of nearness, can be defined using these open sets.

https://en.m.wikipedia.org/wiki/Open_set

https://en.m.wikipedia.org/wiki/Interval_(mathematics)#Terminology

**Abgeschlossene Menge / Intervall**

* **A closed set is a set whose complement is an open set.**

* In a topological space, a closed set can be defined as a set which contains all its limit points. In a complete metric space, a closed set is a set which is closed under the limit operation.

* In a topological space, a set is closed if and only if it coincides with its closure. Equivalently, **a set is closed if and only if it contains all of its limit points**. Yet another equivalent definition is that a set is closed if and only **if it contains all of its boundary points**.

https://en.m.wikipedia.org/wiki/Closed_set

Ein einfaches Beispiel ist das Intervall [0,1] in den reellen Zahlen (mit der Standardtopologie, erzeugt durch die Metrik $d_{x y}=|x-y|$. 

* Das Komplement von [0,1] ist die Vereinigung $(-\infty, 0) \cup(1, \infty)$ zweier offener Intervalle, also eine offene Menge, also ist [0,1] eine abgeschlossene Menge. 

* Deshalb nennt man das Intervall [0,1] ein abgeschlossenes Intervall. 

* **Dagegen ist das Intervall (0,1] nicht abgeschlossen, denn das Komplement $(-\infty, 0] \cup(1, \infty)$ ist
nicht offen.**

https://de.m.wikipedia.org/wiki/Abgeschlossene_Menge

**Abgeschlossenes Intervall**

>$[a, b]:=\{x \in \mathbb{R} \mid a \leq x \leq b\}$

Das Intervall enthält sowohl $a$ als auch $b$.

**Ein Intervall ist genau dann kompakt, wenn es abgeschlossen und beschränkt ist.**

<br>

**Offenes Intervall**

>$(a, b)=] a, b[:=\{x \in \mathbb{R} \mid a<x<b\}$

Das Intervall enthält weder $a$ noch $b$. Die Notation $(a, b)$ ist die traditionell verwendete, während ] $a, b\left[\text { auf Bourbaki zurückgeht. }^{[3]}\right.$
Halboffenes (genauer rechtsoffenes) Intervall

>$[a, b)=[a, b[:=\{x \in \mathbb{R} \mid a \leq x<b\}$

Das Intervall enthält $a$, aber nicht $b$.
Halboffenes (genauer linksoffenes) Intervall

> $(a, b]=\mid a, b]:=\{x \in \mathbb{R} \mid a<x \leq b\}$

Das Intervall enthält nicht $a,$ wohl aber $b$
Im Fall von $a=0$ und $b=1$ heißt $(a, b)$ das offene Einheitsintervall und $[a, b]$ das abgeschlossene Einheitsintervall.

**// Bounded & Unbounded Set (Beschränkt)**

* **Bounded Set = Beschränkte Menge**

Sei a < b. 
* Ein beschränktes Intervall mit der unteren Grenze a und der oberen Grenze b ist abgeschlossen, wenn es beide Grenzen enthält, und offen, wenn beide Grenzen nicht enthalten sind. 
* Ein beschränktes Intervall heißt halboffen, wenn es genau eine der beiden Intervallgrenzen enthält.

* A set is called bounded if it is, in a certain sense, of finite size. Conversely, a set which is not bounded is called unbounded. 

* The word 'bounded' makes no sense in a general topological space without a corresponding metric.

https://en.m.wikipedia.org/wiki/Bounded_set

**Unbeschränktes Intervall**

* Wenn auf einer Seite die Intervallgrenze fehlt, es dort also keine Schranke geben soll, spricht man von einem (auf dieser Seite) unbeschränkten Intervall. 

* Meist werden hierfür die bekannten Symbole −∞ und ∞ als „Ersatz“-Intervallgrenzen verwendet, die selbst nie zum Intervall gehören (deshalb die Schreibung mit runder Klammer). In mancher Literatur werden beschränkte Intervalle auch als eigentlich, unbeschränkte als uneigentlich bezeichnet.

**Infimum und Supremum**

Die Begriffe obere und untere Schranke werden im Artikel Supremum ausführlich beschrieben.

https://de.m.wikipedia.org/wiki/Infimum_und_Supremum

**// = Compactness (Closed & Bounded / Abgeschlossen & Beschränkt)**

![alternativer Text](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7c/Compact.svg/512px-Compact.svg.png)

Per the compactness criteria for Euclidean space (endlich dimensional!) as stated in the Heine-Borel Theorem:

* the interval A = (−∞, −2] is not compact because it is not bounded (but it's closed right side). 

* The interval C = (2, 4) is not compact because it is not closed (but it's bounded). 

* The interval B = [0, 1] is compact because it is both closed and bounded.

Compactness is a property that generalizes the notion of a subset of Euclidean space **being closed** (i.e., containing all its limit points) and **bounded** (i.e., having all its points lie within some fixed distance of each other).

* Examples include a closed interval, a rectangle, or a finite set of points. This notion is defined for more general topological spaces than Euclidean space in various ways.

* importance of including the boundary points of the interval, since the limit points must be in the space itself — an open (or half-open) interval of the real numbers is not compact. 

* It is also crucial that the interval be bounded, since in the interval [0,∞), one could choose the sequence of points 0, 1, 2, 3, ..., of which no sub-sequence ultimately gets arbitrarily close to any given real number.

Nur in endlich dimensionale Räume?

https://en.m.wikipedia.org/wiki/Compact_space

##### **Links**

https://m0nads.wordpress.com/tag/persistent-homology/

https://bastian.rieck.me/blog/posts/2020/icml_topology_roundup/

https://en.wikipedia.org/wiki/Group_homomorphism

https://en.wikipedia.org/wiki/Kernel_(algebra)

https://en.wikipedia.org/wiki/Kernel_(linear_algebra)

https://en.wikipedia.org/wiki/Kernel_(category_theory)

https://en.wikipedia.org/wiki/Homological_algebra

https://en.wikipedia.org/wiki/Induced_homomorphism

https://en.wikipedia.org/wiki/Pushforward_(homology)

http://slideplayer.com/slide/17272227/

## **Homotopy**

##### **Differentiation**

**Measure Theory vs General Topology**

* Although there are superficial similarities, measure theory and general topology have quite different purposes.


* The aim in topology is to define structures that permit us to **define what we mean by a continuous function**. The most common way to do this is to **define a family of subsets of a set which we regard as being open**. The family of open sets must obey two axioms: the union of any set of open sets is open, the intersection of a finite collection of open sets is open. I'll also define a neighbourhood of a point as any superset of any open set containing the point. Then we can say **a function from a space X to a space Y is continuous at x if the inverse image of any neighbourhood of f(x) is a neighbourhood of x**. (Try to see how this generalises epsilon delta definitions.)

* The aim in measure theory is to **study measures of the size of a set such as length, area or volume (for example in vector spaces, or probability in probability spaces)**. Measures must obey certain axioms, the most important one being that the measure of the union of a finite number of disjoint sets is the sum of the measures. Mostly this is also extended to countable unions. It turns out that **it is not usually possible to define a measure on every subset so we define a class of measurable sets**. These obey superficially similar axioms to open sets. A countable union of measurable sets is measurable and the intersection and difference of two measurable sets is measurable. But notice the different restrictions on the number of sets. (It is quite possible, and frequently happens, that a measure is defined on a topological space. Vector spaces are an example.)

* **In measure theory there is a focus on σ-algebras**, i.e. non-empty subcollections of the powerset of a fixed set Ω that are closed under countable unions and complements. **In general topology there is a focus on topologies, i.e. subcollections of the powerset of a fixed set 𝑋 that are closed under finite intersections and arbitrary unions**. They “come together” if you are dealing with σ-algebras that are generated by topologies. Then we meet the **Borel σ-algebras**. If you study general topology then you can do completely without measure theory. If you study measure theory then some (not much) knowledge of topology is required. This especially when the Borel σ-algebras are on the menu.

* Measures deal with metrics and sets, which are commonly defined within a topology. General topology deals with properties of topological spaces, including the behavior of sets and sequences.

https://www.quora.com/What-are-the-main-similarities-and-differences-between-measure-theory-and-general-topology

**Geometry, Topology & Algebraic Topology**

*Geometry*

* The central object of study in plane geometry is congruence. Two shapes are congruent when one can be mapped to the other via a rigid motion: sliding it along, rotating it, or reflecting it. 

* No deformations, expansions, or other twists are allowed. So in geometry we can talk about angles, for example, since angles don't change when you slide and rotate. Congruent triangles are ones that are the same except for a possible translation (sliding), rotation and maybe reflection (taking a mirror image).

*Topology*

* First, we're not limited to simple polygonal shapes in the plane - we're dealing with any "shape" (topological space) for which we have a notion of "nearness". Intuitively it's fine to think of familiar shapes in the plane, space or (if your imagination is up to it) 4 dimensions and beyond. In fact, topology is a lot more permissive than this, and allows for spaces that look like nothing that fits into any number of dimensions.

* The second way topology differs from plane geometry is that the notion of "being the same thing", instead of congruence, is called topological invariance, which is formally defined as a bi-directional mapping of one shape to another which preserves this notion of points being near each other. It's ok to stretch and twist, it's ok to tear and paste back, but it's not ok to make a hole and keep it there, or to otherwise break things apart.

* If you have a rubber band, you can use it to form a circle, an ellipse, or a square. Those are very different shapes geometrically, but from the topological perspective they are the same.

*Algebraic Topology*

* in many ways the permissive nature of topology makes it hard to tell things apart. It's not entirely easy to show that the plane and 3d space are different topologically, which sounds a bit silly but it's a fact. Showing that 3d space isn't 4d space is even harder.

* One of the ways people found to deal with those difficulties is to create gadgets (officially called functors) that map topological spaces into objects that are easier to handle - algebraic objects like vector spaces and groups. If our layperson doesn't know what those are, we can get some headway by just declaring that our gadgets map topological spaces into numbers, or lists of numbers.

* We then show that if two topological spaces are the same (topologically), they will get the same numbers attached to them. Contrariwise, if two spaces are tagged with different numbers, they cannot be the same.

* Defining those gadgets isn't entirely trivial. Classically this was done via things called "homology" and "homotopy" groups. The latter ones can be roughly described as counting the number of distinct ways you can put a closed rubber band inside your space. Two such rubber bands aren't considered distinct if one can be deformed into the other inside your space.



##### **Homotopy Theory**

Eine Homotopie zwischen zwei stetigen Abbildungen $f, g: X \rightarrow Y$
ist eine stetige Abbildung

>$
H: X \times[0,1] \rightarrow Y
$

mit der Eigenschaft

>$
H(x, 0)=f(x) \text { und } H(x, 1)=g(x)
$

wobei [0,1] das Einheitsintervall ist. Der erste Parameter [0] entspricht also dem der ursprünglichen Abbildungen und der **zweite Parameter [1] gibt den Grad der Deformation an**. Besonders anschaulich wird die Definition, wenn man sich den zweiten Parameter als "Zeit" vorstellt.

Man sagt, $f$ sei homotop zu $g$ und schreibt $f \sim g$. Homotopie ist eine **Äquivalenzrelation auf der Menge der stetigen Abbildungen $X \rightarrow Y$**, die zugehörigen Äquivalenzklassen heißen Homotopieklassen, die Menge dieser Äquivalenzklassen wird häufig mit $[X, Y]$ bezeichnet.
Eine stetige Abbildung $f: X \rightarrow Y$ heißt nullhomotop, wenn sie homotop zu einer konstanten Abbildung ist.

**Beispiel**

![xxx](https://upload.wikimedia.org/wikipedia/commons/2/21/Homotopie_Bsp.png)

*Homotopie eines Kreises in R² auf einen Punkt*

Sei $X=S^{1} \subset \mathbb{R}^{2}$ der Einheitskreis in der Ebene und $Y=\mathbb{R}^{2}$ die ganze
Ebene. Die Abbildung $f$ sei die Einbettung von $X$ in $Y$, und $g$ sei die Abbildung, die ganz $X$ auf den Ursprung abbildet, also

>$
f: X \rightarrow Y, f(x)=x \text { und } g: X \rightarrow Y, g(x)=0
$

Dann sind $f$ und $g$ zueinander homotop. Denn $H: X \times[0,1] \rightarrow \mathbb{R}^{2}$ mit $H(x, t)=(1-t) \cdot f(x)$
ist stetig und erfüllt
* $H(x, 0)=1 \cdot f(x)=f(x)$,
* $H(x, 1)=0 \cdot f(x)=0=g(x)$

https://de.m.wikipedia.org/wiki/Homotopie

##### **Homotopy Groups (incl. Fundamental Group)**

* Homotopy groups are used in algebraic topology to classify topological spaces. 

* The first and simplest homotopy group is the fundamental group, which records information about loops in a space. Intuitively, homotopy groups record information about the basic shape, or holes, of a topological space.

* In modern mathematics it is common to study a category by associating to every object of this category a simpler object that still retains sufficient information about the object of interest.

* **Homotopy groups are such a way of associating <u>groups</u> to topological spaces.**

https://de.m.wikipedia.org/wiki/Homotopiegruppe

**Fundamental Group**

* **first and simplest homotopy group** is the fundamental group, which records information about loops in a space. Intuitively, homotopy groups record information about the basic shape, or holes, of a topological space.

* The fundamental group of a topological space is the group of the equivalence classes under homotopy of the loops contained in the space. It records information about the basic shape, or holes, of the topological space. 

* The fundamental group is the first and simplest homotopy group. The fundamental group is a homotopy invariant—topological spaces that are homotopy equivalent (or the stronger case of homeomorphic) have isomorphic fundamental groups.

https://en.m.wikipedia.org/wiki/Fundamental_group

##### **Invariants ('Topological Properties')**

**Allgemeine Invarianten in der Mathematik**

https://de.m.wikipedia.org/wiki/Invariante_(Mathematik)

**Topologische Invarianten**

* Topological invariants are numbers that we assign to a shape, or a simplicial complex in this case, to learn something about its global structure.

* For example Euler characteristic of a simplicial complex in any dimension

* a topological property or topological invariant is a property of a topological space which is invariant under homeomorphisms. 

* That is, a property of spaces is a topological property if whenever a space X possesses that property every space homeomorphic to X possesses that property. Informally, a topological property is a property of the space that can be expressed using open sets.

1. Anzahl der Zusammenhangs-Komponenten (connected components)

2. Isomorphie-Klasse der Fundamentalgruppe

3. Euler-Charakteristik (älter, wurden später durch Betti verallgemeinert)

4. Geschlecht einer Fläche (Genus)

5. Bettizahlen eines topologischen Raumes

Siehe [deutsche Definition](https://de.m.wikipedia.org/wiki/Topologische_Invariante) und [englische Definition](https://en.m.wikipedia.org/wiki/Topological_property) auf Wikipedia

**Geschlecht einer Fläche (Genus)**

https://de.m.wikipedia.org/wiki/Geschlecht_(Fläche)

##### **Homöomorphismus**

* Isomorphism in Topology (in Set it's bijections, in hTop it's called homotopy equivalences)

https://de.m.wikipedia.org/wiki/Homöomorphismus

##### **Weiterführende Links**

https://de.m.wikipedia.org/wiki/Homotopie

https://en.m.wikipedia.org/wiki/Regular_homotopy

https://en.m.wikipedia.org/wiki/Homotopy_principle

https://en.m.wikipedia.org/wiki/Sphere_eversion

https://en.m.wikipedia.org/wiki/Immersion_(mathematics)

https://de.m.wikipedia.org/wiki/Geometrische_Topologie

https://en.m.wikipedia.org/wiki/Ambient_isotopy

**Dual: abgeschlossen**

* Eine Teilmenge des topologischen Raums X, deren Komplement eine offene Menge ist, heißt abgeschlossen. 

* Wenn man die oben formulierte Definition dualisiert und das Wort „offen“ durch „abgeschlossen“ ersetzt (sowie Schnitt und Vereinigung vertauscht), ergibt sich eine gleichwertige Definition des Begriffs „topologischer Raum“ über dessen System abgeschlossener Mengen.

https://de.m.wikipedia.org/wiki/Netz_(Topologie)

https://de.m.wikipedia.org/wiki/Topologische_Algebra

https://en.m.wikipedia.org/wiki/Fibration

https://en.m.wikipedia.org/wiki/Cofibration

https://de.m.wikipedia.org/wiki/Grothendieck-Topologie

https://de.m.wikipedia.org/wiki/Algebraische_Topologie

https://en.m.wikipedia.org/wiki/Characterizations_of_the_category_of_topological_spaces

https://en.m.wikipedia.org/wiki/Category_of_topological_spaces

https://en.m.wikipedia.org/wiki/Topological_space

https://de.m.wikipedia.org/wiki/Triviale_Topologie

https://de.m.wikipedia.org/wiki/Diskrete_Topologie

https://de.m.wikipedia.org/wiki/Geometrisierung_von_3-Mannigfaltigkeiten

https://de.m.wikipedia.org/wiki/Differentialtopologie

https://de.m.wikipedia.org/wiki/Gr%C3%B6bere_und_feinere_Topologien

https://de.m.wikipedia.org/wiki/Axiomatische_Homologie

https://de.m.wikipedia.org/wiki/Simpliziale_Homologie

https://de.m.wikipedia.org/wiki/Homologietheorie

https://de.m.wikipedia.org/wiki/Singuläre_Homologie

https://mathoverflow.net/questions/24671/why-do-the-homology-groups-capture-holes-in-a-space-better-than-the-homotopy-gro

https://de.m.wikipedia.org/wiki/Homologietheorie